# Using CARTOframes to view earthquakes from the last 30 days

In [1]:
import cartoframes
from cartoframes import Credentials
from cartoframes import Layer, BaseMap, styling
import pandas as pd
%matplotlib inline

In [8]:
USERNAME = 'michellemho-carto'  # <-- replace with your username 
APIKEY = 'abcde'       # <-- your CARTO API key
creds = Credentials(username=USERNAME, 
                    key=APIKEY)
creds.save()  # save credentials for later use (not dependent on Python session)
cc = cartoframes.CartoContext(creds)

In [9]:
cc = cartoframes.CartoContext()

In [10]:
cc.creds

Credentials(username=michellemho-carto, key=7c01c3ad9bfe3abea670239c8d8b52ffffdd5177, base_url=https://michellemho-carto.carto.com)

In [11]:
# Use Pandas to read a CSV from the USGS feed of earthquakes from the last 30 days
earthquakes = pd.read_csv('https://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/all_month.csv')
earthquakes.head()

,time,latitude,longitude,depth,mag,magType,nst,gap,dmin,rms,...,updated,place,type,horizontalError,depthError,magError,magNst,status,locationSource,magSource
0,2017-11-09T14:03:55.926Z,64.614300,-149.396800,19.50,1.60,ml,NaN,NaN,NaN,0.66,...,2017-11-09T14:12:34.486Z,"14km WNW of North Nenana, Alaska",earthquake,NaN,0.10,NaN,NaN,automatic,ak,ak
1,2017-11-09T13:27:52.200Z,35.845833,-118.257167,1.69,1.27,ml,15.0,99.0,0.150400,0.18,...,2017-11-09T13:31:25.279Z,"18km NE of Kernville, CA",earthquake,0.40,0.81,0.21,15.0,automatic,ci,ci
2,2017-11-09T13:26:10.467Z,59.781200,-153.322300,126.00,2.80,ml,NaN,NaN,NaN,0.57,...,2017-11-09T13:38:45.397Z,"83km W of Anchor Point, Alaska",earthquake,NaN,0.40,NaN,NaN,automatic,ak,ak
3,2017-11-09T13:13:11.754Z,61.658900,-150.985200,68.00,1.30,ml,NaN,NaN,NaN,0.23,...,2017-11-09T13:16:21.674Z,"51km W of Willow, Alaska",earthquake,NaN,0.80,NaN,NaN,automatic,ak,ak
4,2017-11-09T12:35:49.140Z,38.817501,-122.786667,1.17,0.85,md,8.0,117.0,0.006307,0.02,...,2017-11-09T12:58:02.777Z,"5km W of Cobb, California",earthquake,0.47,0.39,0.07,3.0,automatic,nc,nc


In [12]:
# Write earthquake data to CARTO
cc.write(earthquakes, 'earthquakes_30', lnglat=('longitude','latitude'), overwrite=True)

The following columns were changed in the CARTO copy of this dataframe:
magType -> magtype
horizontalError -> horizontalerror
depthError -> deptherror
magError -> magerror
magNst -> magnst
locationSource -> locationsource
magSource -> magsource
Table successfully written to CARTO: https://michellemho-carto.carto.com/dataset/earthquakes_30


In [13]:
# Map the earthquake data
cc.map(layers=[Layer('earthquakes_30')])

In [14]:
# Style the color and size by the magnitude
cc.map(layers=[Layer('earthquakes_30',
       color={'column': 'mag', 'scheme': styling.redOr(5, bin_method='equal')},
       size={'column':'mag', 'bins':5, 'bin_method':'equal'})])

In [15]:
# Try torque animation of time-stamp column

# Just to make sure time-stamp column is a pandas date type
earthquakes['time'] = pd.to_datetime(earthquakes.time)

In [16]:
earthquakes.head(2)

,time,latitude,longitude,depth,mag,magType,nst,gap,dmin,rms,...,updated,place,type,horizontalError,depthError,magError,magNst,status,locationSource,magSource
0,2017-11-09 14:03:55.926,64.614300,-149.396800,19.50,1.60,ml,NaN,NaN,NaN,0.66,...,2017-11-09T14:12:34.486Z,"14km WNW of North Nenana, Alaska",earthquake,NaN,0.10,NaN,NaN,automatic,ak,ak
1,2017-11-09 13:27:52.200,35.845833,-118.257167,1.69,1.27,ml,15.0,99.0,0.1504,0.18,...,2017-11-09T13:31:25.279Z,"18km NE of Kernville, CA",earthquake,0.4,0.81,0.21,15.0,automatic,ci,ci


In [ ]:
# Update CARTO dataset
cc.write(earthquakes, 'earthquakes_30', lnglat=('longitude','latitude'), overwrite=True)

The following columns were changed in the CARTO copy of this dataframe:
magType -> magtype
horizontalError -> horizontalerror
depthError -> deptherror
magError -> magerror
magNst -> magnst
locationSource -> locationsource
magSource -> magsource


In [ ]:
# Create an animated map by specifying the time of the layer
cc.map(layers=[Layer('earthquakes_30', time = {'column':'time', 'method':'sum', 'trails':4, 'duration':30})])